In [1]:
# !pip install opencv-python mediapipe

In [2]:
# ✅ Import libraries
import cv2
import mediapipe as mp
import random
import time

In [3]:
# ✅ Initialize MediaPipe and Webcam
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.75)

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Webcam not detected!")
    exit()

In [4]:
# ✅ Gesture definitions and helper functions
gestures = {
    "rock": [0, 0, 0, 0, 0],
    "paper": [1, 1, 1, 1, 1],
    "scissors": [0, 1, 1, 0, 0]
}

def get_finger_state(hand_landmarks):
    finger_states = []
    tips = [8, 12, 16, 20]
    for tip in tips:
        finger_states.append(int(hand_landmarks.landmark[tip].y < hand_landmarks.landmark[tip - 2].y))
    thumb = int(hand_landmarks.landmark[4].x > hand_landmarks.landmark[2].x)
    finger_states.insert(0, thumb)
    return finger_states

def detect_gesture(finger_states):
    for name, pattern in gestures.items():
        if pattern == finger_states:
            return name
    return "unknown"

def get_ai_move():
    return random.choice(list(gestures.keys()))

def get_winner(user, ai):
    if user == ai:
        return "Draw"
    elif (user == "rock" and ai == "scissors") or          (user == "paper" and ai == "rock") or          (user == "scissors" and ai == "paper"):
        return "You Win!"
    else:
        return "You Lose!" 

In [ ]:
# ✅ Game Loop
last_match_time = time.time()
match_interval = 5
your_move = "None"
ai_move = "None"
outcome = ""

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)
    current_time = time.time()

    if result.multi_hand_landmarks:
        hand_landmarks = result.multi_hand_landmarks[0]
        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        if current_time - last_match_time >= match_interval:
            finger_state = get_finger_state(hand_landmarks)
            your_move = detect_gesture(finger_state)
            ai_move = get_ai_move()
            outcome = get_winner(your_move, ai_move)
            last_match_time = current_time
            time.sleep(2)  # 2 seconds pause after move

    cv2.putText(frame, f"Your Move: {your_move.upper()}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"AI Move: {ai_move.upper()}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    if current_time - last_match_time < 5:
        cv2.putText(frame, f"Result: {outcome}", (10, 110),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    countdown = max(0, int(match_interval - (current_time - last_match_time)))
    cv2.putText(frame, f"Next Match In: {countdown}s", (10, 150),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 255, 255), 2)

    cv2.imshow("Rock-Paper-Scissors", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()